<a href="https://colab.research.google.com/github/12003586/Hackaton/blob/main/Hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random

from os import listdir
from os.path import isfile, join

import urllib.request
import requests
from io import BytesIO

from PIL import Image
from PIL.ImageEnhance import Color, Contrast, Brightness

#from tensorflow import keras
from tensorflow.keras.models import Model as tf_Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras import optimizers as tf_optim

from sklearn.ensemble import AdaBoostRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, OneHotEncoder

import sklearn.metrics as metrics

from imageai.Classification.Custom import ClassificationModelTrainer

from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360" -d "/content"

warning [/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #90487).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

image_shape = (224, 224, 3)

base_model = VGG16(input_shape = image_shape,
    include_top = False,
    weights = 'imagenet')

base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
n_outputs = 10

x = Conv2D(filters = 1024, kernel_size=(2,2), padding='valid')(base_model.output)
x = Conv2D(filters = 1024, kernel_size=(2,2), padding='same')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dropout(rate=0.25)(x)
x = Dense(128, activation='relu')(x)
x = Dense(n_outputs, activation='sigmoid')(x)

model = tf_Model(base_model.input, x, name = 'expanded_model')

model.summary()

Model: "expanded_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)    

In [7]:
model.compile(optimizer = tf_optim.RMSprop(learning_rate=0.0001), loss = "mean_scuared_error", metrics = ["mean_scuared_error"])


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a data generator
datagen_train = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
)  # we don't expect Bo to be upside-down so we will not flip vertically



In [12]:
train_it = datagen.flow_from_directory('content/Kaggle_Fruits360/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('content/Kaggle_Fruits360/', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

NameError: ignored